In [353]:
import csv
import re
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


def remover_tempo_morto(dados_do_eletrodo):
    condutividade_inicial = [cn for cn in dados_do_eletrodo['condutividade_normalizada'] if abs(cn) <= 1e-2]
    pontos_removidos = len(condutividade_inicial)
    tempo = dados_do_eletrodo['tempo'][:-pontos_removidos]
    condutividade_normalizada = dados_do_eletrodo['condutividade_normalizada'][pontos_removidos:]
    return tempo, condutividade_normalizada

def logaritmo_da_variancia(dados_do_caso):
    numero_de_pontos = min([len(eletrodo['condutividade_normalizada']) for eletrodo in dados_do_caso.values()])
    pontos = [eletrodo['tempo'] for eletrodo in dados_do_caso.values()][0][0:numero_de_pontos]
    log_var = [list((np.array(eletrodo['condutividade_normalizada'][0:numero_de_pontos]) - 1)**2) for eletrodo in dados_do_caso.values()]
    log_var = list(np.log10(np.mean(np.array(log_var), axis=0)))
    return (pontos, log_var)

def obter_dados(casos, eletrodos, tempo_morto = False):
    dados_dos_casos = dict()
    for caso in casos:
        dados_do_caso = dict()
        for eletrodo in eletrodos:
            prefixo_eletrodo = re.search('caso_\d+_', caso).group()
            nome_do_arquivo = f'dados_do_condutivimetro/{caso}/{prefixo_eletrodo}{eletrodo}.csv'
            with open(nome_do_arquivo, 'r') as arquivo:
                dados_csv = csv.reader(arquivo, delimiter=';')
                next(dados_csv)
                dados_lista = [linha[0:4] for linha in dados_csv]
                data = [i[0].strip() for i in dados_lista]
                horario = [i[1].strip() for i in dados_lista]
                condutividade = [float(i[2].strip().replace(',', '.')) for i in dados_lista]
                temperatura = [float(i[3].strip().replace(',', '.')) for i in dados_lista]
                arquivo.close()
            dados_do_eletrodo = {'data': data,
                                'horario': horario,
                                'condutividade': condutividade,
                                'temperatura': temperatura}
            dados_do_eletrodo['numero_de_registros'] = len(condutividade)
            t0 = datetime.strptime(f'{data[0]} {horario[0]}', '%d/%m/%Y %H:%M:%S')
            t1 = datetime.strptime(f'{data[1]} {horario[1]}', '%d/%m/%Y %H:%M:%S')
            t = t1 - t0
            dados_do_eletrodo['intervalo'] = t.seconds
            dados_do_eletrodo['tempo'] = list(range(0,
                                                    dados_do_eletrodo['intervalo'] * dados_do_eletrodo['numero_de_registros'],
                                                    dados_do_eletrodo['intervalo']))
            dados_do_eletrodo['limites_condutividade'] = (condutividade[0], condutividade[-1])
            c0, cinf = dados_do_eletrodo['limites_condutividade']
            dados_do_eletrodo['condutividade_normalizada'] = [(c - c0) / (cinf - c0) for c in condutividade]
            if tempo_morto:
                dados_do_eletrodo['tempo'], dados_do_eletrodo['condutividade_normalizada'] = remover_tempo_morto(dados_do_eletrodo)
            dados_do_caso[eletrodo] = dados_do_eletrodo
        dados_do_caso['logaritmo_da_variancia'] = logaritmo_da_variancia(dados_do_caso)
        dados_dos_casos[caso] = dados_do_caso
    return dados_dos_casos

def exportar_dados(dados_dos_casos):
    for dados_key, dados_value in dados_dos_casos.items():
        for casos_key, casos_value in dados_value.items():
            prefixo_eletrodo = re.search('caso_\d+_', dados_key).group()
            if casos_key.startswith('logaritmo'):
                nome_do_arquivo = f'dados_do_condutivimetro/{dados_key}/{prefixo_eletrodo}logaritmo_da_variancia.dat'
                with open(nome_do_arquivo, 'w') as arquivo:
                    pontos, log_var = casos_value
                    for p, lv in zip(pontos, log_var):
                        linha = f'{p:d} {lv:.6f}\n'
                        if 'inf' in linha:
                            continue
                        arquivo.write(linha)
                    arquivo.close
            else:
                nome_do_arquivo = f'dados_do_condutivimetro/{dados_key}/{prefixo_eletrodo}{casos_key}.dat'
                with open(nome_do_arquivo, 'w') as arquivo:
                    tempo = casos_value['tempo']
                    condutividade_normalizada = casos_value['condutividade_normalizada']
                    for t, cn in zip(tempo, condutividade_normalizada):
                        linha = f'{t:d} {cn:.6f}\n'
                        arquivo.write(linha)
                    arquivo.close


In [354]:
casos = ['caso_1_N_200rpm_beta_0', 'caso_2_N_600rpm_beta_0', 'caso_3_N_600rpm_beta_15', 'caso_4_N_600rpm_beta_0']
eletrodos = ['eletrodo_1_fundo', 'eletrodo_2_topo']
dados = obter_dados(casos, eletrodos, True)
# t = dados[casos[0]][eletrodos[0]]['condutividade_normalizada_corrigida']
# print(len(t), t)
exportar_dados(dados)

C:\Users\Gilberto\AppData\Local\Temp\ipykernel_11984\366863850.py:19: RuntimeWarning: divide by zero encountered in log10
  log_var = list(np.log10(np.mean(np.array(log_var), axis=0)))
